# Introduction to Generative AI

Generative AI is a type of artificial intelligence that can create new content and ideas, including conversations, stories, images, videos, and music. Like all artificial intelligence, generative AI is powered by machine learning models—very large models that are pre-trained on vast amounts of data and commonly referred to as Foundation Models (FMs). Apart from content creation, generative AI is also used to improve the quality of digital images, edit video, build prototypes quickly for manufacturing, augment data with synthetic datasets, and more.

## Using with Falcon model 

---
In this demo notebook, we demonstrate how to use the SageMaker Python SDK to deploy Falcon models for text generation. It is a permissively licensed ([Apache-2.0](https://jumpstart-cache-prod-us-east-2.s3.us-east-2.amazonaws.com/licenses/Apache-License/LICENSE-2.0.txt)) open source model trained on the [RefinedWeb dataset](https://huggingface.co/datasets/tiiuae/falcon-refinedweb). We show several example use cases including code generation, question answering, translation etc.

---

### About the model

---
Falcon is a causal decoder-only model built by [Technology Innovation Institute](https://www.tii.ae/) (TII) and trained on more than 1 trillion tokens of RefinedWeb enhanced with curated corpora. It was built using custom-built tooling for data pre-processing and model training built on Amazon SageMaker. It features an architecture optimized for inference, with FlashAttention and multiquery. 


[Refined Web Dataset](https://huggingface.co/datasets/tiiuae/falcon-refinedweb): Falcon RefinedWeb is a massive English web dataset built by TII and released under an Apache 2.0 license. It is a highly filtered dataset with large scale de-duplication of CommonCrawl. It is observed that models trained on RefinedWeb achieve performance equal to or better than performance achieved by training model on curated datasets, while only relying on web data.

**Model Sizes:**
- **Falcon-7b**: It is a 7 billion parameter model trained on 1.5 trillion tokens.
- **Falcon-40B**: It is a 40 billion parameter model trained on 1 trillion tokens.  It has surpassed renowned models like LLaMA-65B, StableLM, RedPajama and MPT on the public leaderboard maintained by Hugging Face, demonstrating its exceptional performance without specialized fine-tuning. To see comparison, see [OpenLLM Leaderboard](https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard). 

**Instruct models (Falcon-7b-instruct/Falcon-40B-instruct):** Instruct models are base falcon models fine-tuned on a mixture of chat and instruction datasets. They are ready-to-use chat/instruct models.  To use these models, please select `model_id` in the cell above to be "huggingface-textgeneration-falcon-7b-instruct-bf16" or "huggingface-textgeneration-falcon-40b-instruct-bf16".

It is [recommended](https://huggingface.co/tiiuae/falcon-7b) that Instruct models should be used without fine-tuning and base models should be fine-tuned further on the specific task.

**Limitations:**

- Falcon models are mostly trained on English data and may not generalize to other languages. 
- Falcon carries the stereotypes and biases commonly encountered online and in the training data. Hence, it is recommended to develop guardrails and to take appropriate precautions for any production use. This is a raw, pretrained model, which should be further finetuned for most usecases.


## Step 1 : Upgrade SageMaker 

In [2]:
!pip install sagemaker --quiet --upgrade --force-reinstall

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.6.0 requires daal==2021.4.0, which is not installed.
spyder 5.3.3 requires pyqt5<5.16, which is not installed.
spyder 5.3.3 requires pyqtwebengine<5.16, which is not installed.
awscli 1.31.9 requires botocore==1.33.9, but you have botocore 1.34.56 which is incompatible.
awscli 1.31.9 requires s3transfer<0.9.0,>=0.8.0, but you have s3transfer 0.10.0 which is incompatible.
distributed 2022.7.0 requires tornado<6.2,>=6.0.3, but you have tornado 6.4 which is incompatible.
jupyterlab 3.4.4 requires jupyter-server~=1.16, but you have jupyter-server 2.12.1 which is incompatible.
jupyterlab-server 2.10.3 requires jupyter-server~=1.4, but you have jupyter-server 2.12.1 which is incompatible.
notebook 6.5.6 requires jupyter-client<8,>=5.3.4, but you have jupyter-client 8.6.0 which is incompatible.
notebook 6.5.

## Step 2 : Define the Model
Define the **huggingface-llm-falcon-7b-instruct-bf16** model

In [3]:
model_id, model_version, = (
    "huggingface-llm-falcon-7b-instruct-bf16",
    "*",
)

## Step 3 : Deploy the model in Amazon SageMaker Inference Endpoint.


In [4]:
%%time
from sagemaker.jumpstart.model import JumpStartModel

my_model = JumpStartModel(model_id=model_id,instance_type="ml.g5.2xlarge" )
predictor = my_model.deploy()

/opt/conda/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/opt/conda/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


Using model 'huggingface-llm-falcon-7b-instruct-bf16' with wildcard version identifier '*'. You can pin to version '2.2.1' for more stable results. Note that models may have different input/output signatures after a major version upgrade.


-----------------!CPU times: user 1.49 s, sys: 478 ms, total: 1.97 s
Wall time: 9min 4s


## Step 4 : Test the deployed model

The model supports a variety of actions like code generation, sentiment analysis, question answering, and summarization.

### Supported parameters

***
Some of the supported parameters while performing inference are the following:

* **max_length:** Model generates text until the output length (which includes the input context length) reaches `max_length`. If specified, it must be a positive integer.
* **max_new_tokens:** Model generates text until the output length (excluding the input context length) reaches `max_new_tokens`. If specified, it must be a positive integer.
* **num_beams:** Number of beams used in the greedy search. If specified, it must be integer greater than or equal to `num_return_sequences`.
* **no_repeat_ngram_size:** Model ensures that a sequence of words of `no_repeat_ngram_size` is not repeated in the output sequence. If specified, it must be a positive integer greater than 1.
* **temperature:** Controls the randomness in the output. Higher temperature results in output sequence with low-probability words and lower temperature results in output sequence with high-probability words. If `temperature` -> 0, it results in greedy decoding. If specified, it must be a positive float.
* **early_stopping:** If True, text generation is finished when all beam hypotheses reach the end of sentence token. If specified, it must be boolean.
* **do_sample:** If True, sample the next word as per the likelihood. If specified, it must be boolean.
* **top_k:** In each step of text generation, sample from only the `top_k` most likely words. If specified, it must be a positive integer.
* **top_p:** In each step of text generation, sample from the smallest possible set of words with cumulative probability `top_p`. If specified, it must be a float between 0 and 1.
* **return_full_text:** If True, input text will be part of the output generated text. If specified, it must be boolean. The default value for it is False.
* **stop**: If specified, it must a list of strings. Text generation stops if any one of the specified strings is generated.

We may specify any subset of the parameters mentioned above while invoking an endpoint. 

For more parameters and information on HF LLM DLC, please see [this article](https://huggingface.co/blog/sagemaker-huggingface-llm#4-run-inference-and-chat-with-our-model).
***

### Step 4.1: Test endpoint

In [5]:
%%time


prompt = "Tell me about Amazon SageMaker."

payload = {
    "inputs": prompt,
    "parameters": {
        "do_sample": True,
        "top_p": 0.9,
        "temperature": 0.8,
        "max_new_tokens": 1024,
        "stop": ["<|endoftext|>", "</s>"]
    }
}

response = predictor.predict(payload)
print(response[0]["generated_text"])


Amazon SageMaker is a cloud-based service that enables machine learning and artificial intelligence (AI) capabilities in applications and services. It allows organizations to train and deploy machine learning models quickly and easily without the need for specialized data scientists.
CPU times: user 11.2 ms, sys: 2.74 ms, total: 14 ms
Wall time: 1.71 s


### Step 4.2 : Test use cases

Let's define a function to query endpoint.

In [6]:
def query_endpoint(payload):
    """Query endpoint and print the response"""
    response = predictor.predict(payload)
    print(f"\033[1m Input:\033[0m {payload['inputs']}")
    print(f"\033[1m Output:\033[0m {response[0]['generated_text']}")

### Step 4.2.1 : Test code generation

In [7]:

payload = {"inputs": "Write a program to compute factorial in python:", "parameters":{"max_new_tokens": 200}}
query_endpoint(payload)

 Input: Write a program to compute factorial in python:
 Output: 
Here is a Python program to compute factorial:

```python
def factorial(n):
    if n == 0:
        return 1
    else:
        return n * factorial(n-1)

print(factorial(5)) # Output: 120
```


### Step 4.2.2 : Test sentence completion

In [8]:
payload = {
    "inputs": "Building a website can be done in 10 simple steps:",
    "parameters":{
        "max_new_tokens": 110,
        "no_repeat_ngram_size": 3
        }
}
query_endpoint(payload)

 Input: Building a website can be done in 10 simple steps:
 Output: 
1. Choose a domain name
2. Register a domain name
3. Choose a web hosting provider
4. Create a website design
5. Add content to your website
6. Test your website
7. Optimize your website for search engines
8. Promote your website
9. Update your website regularly
10. Monitor your website for security


### Step 4.2.3 : Test translation

In [9]:
payload = {
    "inputs": """Translate English to French:

    sea otter => loutre de mer

    peppermint => menthe poivrée

    plush girafe => girafe peluche

    cheese =>""",
    "parameters":{
        "max_new_tokens": 3
    }
}

query_endpoint(payload)

 Input: Translate English to French:

    sea otter => loutre de mer

    peppermint => menthe poivrée

    plush girafe => girafe peluche

    cheese =>
 Output:  fromage

   


### Step 4.2.4 : Test sentiment analysis

In [10]:
payload = {
    "inputs": """"I hate it when my phone battery dies."
                Sentiment: Negative
                ###
                Tweet: "My day has been :+1:"
                Sentiment: Positive
                ###
                Tweet: "This is the link to the article"
                Sentiment: Neutral
                ###
                Tweet: "This new music video was incredibile"
                Sentiment:""",
    "parameters": {
        "max_new_tokens":2
    }
}
query_endpoint(payload)

 Input: "I hate it when my phone battery dies."
                Sentiment: Negative
                ###
                Tweet: "My day has been :+1:"
                Sentiment: Positive
                ###
                Tweet: "This is the link to the article"
                Sentiment: Neutral
                ###
                Tweet: "This new music video was incredibile"
                Sentiment:
 Output:  Positive
                


### Step 4.2.5 : Test question answering

In [11]:
payload = {
    "inputs": "Could you remind me when was the C programming language invented?",
    "parameters":{
        "max_new_tokens": 50
    }
}
query_endpoint(payload)

 Input: Could you remind me when was the C programming language invented?
 Output: 
The C programming language was invented in 1972 by Dennis Ritchie at Bell Labs.


### Step 4.2.6 : Test recipe generation

In [12]:
payload = {"inputs": "What is the recipe for a delicious lemon cheesecake?", "parameters":{"max_new_tokens": 400}}
query_endpoint(payload)

 Input: What is the recipe for a delicious lemon cheesecake?
 Output: 
Here is a recipe for a delicious lemon cheesecake:

Ingredients:
- 1 1/2 cups graham cracker crumbs
- 4 tablespoons butter, melted
- 2 (8 ounce) packages cream cheese, softened
- 1/2 cup granulated sugar
- 2 eggs
- 1/2 cup lemon juice
- 1/2 teaspoon salt
- 1/2 teaspoon vanilla extract
- 1/2 cup heavy cream
- 1/2 cup granulated sugar
- 1/2 teaspoon lemon zest

Instructions:
1. Preheat oven to 350 degrees F.
2. In a medium bowl, mix together the graham cracker crumbs and melted butter. Press the mixture onto the bottom and sides of a 9-inch springform pan.
3. In a large bowl, beat the cream cheese and sugar until smooth. Add the eggs, lemon juice, salt, vanilla, and heavy cream. Beat until well combined.
4. Pour the mixture into the prepared pan.
5. Bake for 30 minutes or until the cheesecake is set.
6. Let cool for 10 minutes before serving.
7. In a small bowl, mix together the lemon zest and sugar. Sprinkle over the

### Step 4.2.7 : Test summarization

In [13]:
payload = {
    "inputs":"""Amazon SageMaker is a fully managed machine learning service. With SageMaker, 
    data scientists and developers can quickly and easily build and train machine learning models, 
    and then directly deploy them into a production-ready hosted environment. It provides an 
    integrated Jupyter authoring notebook instance for easy access to your data sources for 
    exploration and analysis, so you don't have to manage servers. It also provides common 
    machine learning algorithms that are optimized to run efficiently against extremely 
    large data in a distributed environment. With native support for bring-your-own-algorithms 
    and frameworks, SageMaker offers flexible distributed training options that adjust to your 
    specific workflows. Deploy a model into a secure and scalable environment by launching it 
    with a few clicks from SageMaker Studio or the SageMaker console. Summarize the article above:""",
    "parameters":{
        "max_new_tokens":200
        }
    }
query_endpoint(payload)

 Input: Amazon SageMaker is a fully managed machine learning service. With SageMaker, 
    data scientists and developers can quickly and easily build and train machine learning models, 
    and then directly deploy them into a production-ready hosted environment. It provides an 
    integrated Jupyter authoring notebook instance for easy access to your data sources for 
    exploration and analysis, so you don't have to manage servers. It also provides common 
    machine learning algorithms that are optimized to run efficiently against extremely 
    large data in a distributed environment. With native support for bring-your-own-algorithms 
    and frameworks, SageMaker offers flexible distributed training options that adjust to your 
    specific workflows. Deploy a model into a secure and scalable environment by launching it 
    with a few clicks from SageMaker Studio or the SageMaker console. Summarize the article above:
 Output:  SageMaker is a cloud-based machine learning platf

### 5. Clean up the endpoint - Do not run below codes.

In [14]:
# Delete the SageMaker endpoint
# predictor.delete_model()
# predictor.delete_endpoint()